In [1]:
!pip install flask transformers torch


In [2]:
from flask import Flask, request, jsonify, render_template
from transformers import AutoTokenizer, AutoModel, MarianMTModel, MarianTokenizer
import torch

app = Flask(_name_)

# Load models and tokenizers
style_model_name = "bert-base-uncased"
translation_model_name = "Helsinki-NLP/opus-mt-en-es"
style_tokenizer = AutoTokenizer.from_pretrained(style_model_name)
style_model = AutoModel.from_pretrained(style_model_name)
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
translation_model = MarianMTModel.from_pretrained(translation_model_name)

def extract_style_vector(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    style_vector = torch.mean(embeddings, dim=1)
    return style_vector

def translate_text(text, model, tokenizer):
    translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))
    translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return translated_text[0]

def validate_style(original_text, translated_text, original_style_vector, model, tokenizer):
    translated_style_vector = extract_style_vector(translated_text, model, tokenizer)
    similarity = torch.nn.functional.cosine_similarity(original_style_vector, translated_style_vector)
    return similarity.item()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process', methods=['POST'])
def process():
    original_text = request.form['original_text']

    # Extract style vector
    original_style_vector = extract_style_vector(original_text, style_model, style_tokenizer)

    # Translate text
    translated_text = translate_text(original_text, translation_model, translation_tokenizer)

    # Validate style transfer
    similarity_score = validate_style(original_text, translated_text, original_style_vector, style_model, style_tokenizer)

    response = {
        'original_text': original_text,
        'translated_text': translated_text,
        'similarity_score': similarity_score
    }

    return jsonify(response)

if _name_ == '_main_':
    app.run(debug=True)

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\91965\anaconda3\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.